In [1]:
import os
from core import get_empenhos_by_proc
from core.exceptions import ProcNumError

## Testes

In [2]:
assert isinstance(get_empenhos_by_proc('6018.2024/0025416-0', mes=12, ano=2024), list)

In [3]:
try:
    get_empenhos_by_proc('Nao sou um proc')
except ProcNumError as e:
    assert str(e) == 'Número de processo inválido: Nao sou um proc'
else:
    assert False, "ProcNumError was not raised"


In [4]:
resp_exemplo = get_empenhos_by_proc('6018.2024/0025416-0', mes=12, ano=2024)

In [5]:
resp_exemplo = resp_exemplo[0]
resp_exemplo

{'codEmpresa': '01',
 'nomEmpresa': 'PREFEITURA DO MUNICÍPIO DE SÃO PAULO',
 'numReserva': 23689,
 'codEmpenho': 36391,
 'anoEmpenho': 2024,
 'mesEmpenho': 3,
 'datEmpenho': '13/03/2024 00:00:00',
 'codProcesso': '6018202400254160',
 'numCpfCnpj': '36157982000177',
 'txtRazaoSocial': 'REP ENGENHARIA E SERVICOS LTDA',
 'numContrato': 7254,
 'anoContrato': 2024,
 'codOrgao': '07',
 'txDescricaoOrgao': 'Fundo Municipal de Desenvolvimento Social',
 'codUnidade': '10',
 'txDescricaoUnidade': 'Fundo Municipal de Desenvolvimento Social',
 'codFuncao': '10',
 'txDescricaoFuncao': 'Saúde',
 'codSubFuncao': '301',
 'txDescricaoSubFuncao': 'Atenção Básica',
 'codPrograma': '3003',
 'txDescricaoPrograma': 'Ações e Serviços da Saúde em Atenção Básica, Especialidades e Vigilânc',
 'codProjetoAtividade': '1525',
 'txDescricaoProjetoAtividade': 'Construção e Implantação de Equipamentos de Atenção Básica e Especiali',
 'codCategoria': '4',
 'txDescricaoCategoriaEconomica': 'Despesas de Capital',
 'codG

## Parser

In [6]:
resp_exemplo.keys()

dict_keys(['codEmpresa', 'nomEmpresa', 'numReserva', 'codEmpenho', 'anoEmpenho', 'mesEmpenho', 'datEmpenho', 'codProcesso', 'numCpfCnpj', 'txtRazaoSocial', 'numContrato', 'anoContrato', 'codOrgao', 'txDescricaoOrgao', 'codUnidade', 'txDescricaoUnidade', 'codFuncao', 'txDescricaoFuncao', 'codSubFuncao', 'txDescricaoSubFuncao', 'codPrograma', 'txDescricaoPrograma', 'codProjetoAtividade', 'txDescricaoProjetoAtividade', 'codCategoria', 'txDescricaoCategoriaEconomica', 'codGrupo', 'txDescricaoGrupoDespesa', 'codModalidade', 'txDescricaoModalidade', 'codElemento', 'txDescricaoElemento', 'codFonteRecurso', 'txDescricaoFonteRecurso', 'codItemDespesa', 'txDescricaoItemDespesa', 'codSubElemento', 'txDescricaoSubElementoDespesa', 'valTotalEmpenhado', 'valAnuladoEmpenho', 'valEmpenhadoLiquido', 'valLiquidado', 'valPagoExercicio', 'valPagoRestos', 'anexos', 'codReferencia', 'codDestinacaoRecurso', 'codVinculacaoRecurso', 'codExeFonte', 'numeroOriginalContrato'])

In [7]:
for key in resp_exemplo.keys():
    if key.startswith('cod'):
        print(key)

codEmpresa
codEmpenho
codProcesso
codOrgao
codUnidade
codFuncao
codSubFuncao
codPrograma
codProjetoAtividade
codCategoria
codGrupo
codModalidade
codElemento
codFonteRecurso
codItemDespesa
codSubElemento
codReferencia
codDestinacaoRecurso
codVinculacaoRecurso
codExeFonte


In [8]:
class EmpenhoParser:

    chaves_dotacao = [
        'codOrgao',
        'codUnidade',
        'codFuncao',
        'codSubFuncao',
        'codPrograma',
        'codProjetoAtividade',
        'codCategoria',
        'codGrupo',
        'codModalidade',
        'codElemento',
        'codFonteRecurso',
    ]
    
    def parse_cod_empenho(self, empenho: dict) -> str:
        return empenho['codEmpenho']
    
    def parse_empenhado_liquido(self, empenho: dict) -> float:
        return float(empenho['valEmpenhadoLiquido'])
    
    def parse_liquidado(self, empenho: dict) -> float:
        return float(empenho['valLiquidado'])
    
    def parse_pago_restos_pagar(self, empenho: dict) -> float:
        return float(empenho['valPagoRestos'])
    
    def build_dotacao(self, empenho:dict)->str:
        
        dotacao_parts = []
        for chave in self.chaves_dotacao:
            dotacao_parts.append(empenho[chave])
        
        dotacao = '.'.join(dotacao_parts)
        return dotacao
    

    def pipeline(self, empenho: dict) -> dict:

        cod_empenho = self.parse_cod_empenho(empenho)
        empenhado_liquido = self.parse_empenhado_liquido(empenho)
        liquidado = self.parse_liquidado(empenho)
        pago_restos_pagar = self.parse_pago_restos_pagar(empenho)
        dotacao = self.build_dotacao(empenho)
        
        return {
            'cod_empenho': cod_empenho,
            'empenhado_liquido': empenhado_liquido,
            'liquidado': liquidado,
            'pago_restos_pagar': pago_restos_pagar,
            'dotacao': dotacao,
        }
    
    def __call__(self, empenho: dict) -> dict:
        return self.pipeline(empenho)

In [9]:
parse_empenho = EmpenhoParser()
parse_empenho(resp_exemplo)

{'cod_empenho': 36391,
 'empenhado_liquido': 936308.59,
 'liquidado': 762916.47,
 'pago_restos_pagar': 274454.53,
 'dotacao': '07.10.10.301.3003.1525.4.4.90.51.08.2.759.1224'}

## Pipeline

In [11]:
import pandas as pd
from core.utils.date import is_current_year, current_month
from config import DATA_DIR
from tqdm import tqdm
from core.exceptions import ProcNumError, SofRespError

In [12]:
df = pd.read_excel(os.path.join(DATA_DIR, 'original_data.xlsx'))
df.head()

,obra_id,categoria,processo_sei
0,9024,Manual,6018.2024/0025416-0
1,17331,Manual,6012.2025/0003385-5
2,1018,Manual,6019.2022/0001881-6
3,1074,Manual,7910.2022/0000506-0
4,6196,Manual,6022.2019/0006036-8


In [13]:
df = df[['obra_id', 'processo_sei']]

In [14]:
anos = [2025]

In [15]:
parsed_data = []
for ano in anos:
    print(ano)
    for i, row in tqdm(df.iterrows()):
        obra_id = row['obra_id']
        proc_num = row['processo_sei']
        parsed = {
            'obra_id': obra_id,
            'processo': proc_num,
            'ano': ano,
        }
        if is_current_year(ano):
            mes = current_month()
        else:
            mes = 12
        parsed['mes'] = mes
        try:
            empenhos = get_empenhos_by_proc(parsed['processo'], parsed['mes'], parsed['ano'])
        except ProcNumError as e:
            parsed['status'] = 'Número do processo fora do padrão: {}'.format(str(e))
            parsed_data.append(parsed)
            continue
        except SofRespError as e:
            parsed['status'] = 'Erro ao consultar o SOF: {}'.format(str(e))
            parsed_data.append(parsed)
            continue

        if not empenhos:
            parsed['status'] = 'Nenhum empenho encontrado'
            parsed_data.append(parsed)
            continue
        else:
            for empenho in empenhos:
                parsed_empenho = parse_empenho(empenho)
                parsed_empenho.update(parsed)
                parsed_empenho['status'] = 'Sucesso'

                parsed_data.append(parsed_empenho) 
    

2025


4487it [22:24,  3.34it/s]


In [16]:
df = pd.DataFrame(parsed_data)
df.head()

,cod_empenho,empenhado_liquido,liquidado,pago_restos_pagar,dotacao,obra_id,processo,ano,mes,status
0,44513.0,320000.00,320000.00,0.0,07.10.10.301.3003.1525.4.4.90.51.08.1.759.1224,9024,6018.2024/0025416-0,2025,12,Sucesso
1,46072.0,337158.75,337158.75,0.0,84.10.10.304.3027.1530.4.4.90.51.00.1.500.9001,9024,6018.2024/0025416-0,2025,12,Sucesso
2,69897.0,254317.50,247180.06,0.0,84.10.10.304.3027.1530.4.4.90.51.00.1.500.0003,9024,6018.2024/0025416-0,2025,12,Sucesso
3,93744.0,261411.89,261411.89,0.0,84.10.10.304.3027.1530.4.4.90.51.00.1.500.9001,9024,6018.2024/0025416-0,2025,12,Sucesso
4,139865.0,3526546.57,0.00,0.0,07.10.10.301.3003.1525.4.4.90.51.10.1.755.1224,9024,6018.2024/0025416-0,2025,12,Sucesso


In [18]:
if not os.path.exists('dados_salvos'):
    os.mkdir('dados_salvos')

In [ ]:
df.to_excel('dados_salvos/dados_final.xlsx')

## juntando arquivos que foram salvos ao longo da extração

In [20]:
df_20 = pd.read_excel('dados_salvos/18_20.xlsx')

In [21]:
df_21_24 = pd.read_excel('dados_salvos/21_24.xlsx')

In [22]:
df_25 = pd.read_excel('dados_salvos/25.xlsx')

In [23]:
df_20['ano'].unique()

array([2018, 2019, 2020, 2021])

In [25]:
df_20 = df_20[df_20['ano']!=2021].reset_index(drop=True)

In [26]:
df_21_24['ano'].unique()

array([2021, 2022, 2023, 2024, 2025])

In [27]:
df_21_24 =df_21_24[df_21_24['ano']!=2025].reset_index(drop=True)

In [28]:
df_25['ano'].unique()

array([2025])

In [29]:
df_final = [
    df_20,
    df_21_24,
    df_25
]

In [31]:
df_final = pd.concat(df_final)

In [32]:
df_final.to_excel('dados_salvos/extracao_final.xlsx')